## Homework 3, CS678 Spring 2024

### This homework has two submission deadlines: (1) Checkpoint 1 on March 18, 2024, and (2) Checkpoint 2 on March 29, 2024.

Submit the report to Gradescope with the naming convention of
**John_Doe_HW3_Report_CS678_S24_ckpt1/2.pdf** if your name is John Doe and you are submitting for checkpoint 1/2.
Only the report should be submitted to Gradescope. The rest of the code and data files,
including this notebook in its completion form,
should be submitted to Blackboard in a single zipped folder.

<!-- #### IMPORTANT:

After copying this notebook to your Google Drive, please paste a link to it
below. To get a publicly-accessible link, hit the *Share* button at the top
right, then click "Get shareable link" and copy over the result.If you fail to
do this, you will receive no credit for this homework!

***LINK:***

--- -->


##### *How to submit this problem set:*
- Write all the answers in this notebook.

- When creating your final version of the notebook to hand in,
  please do a fresh restart and execute every cell in order.
  One handy way to do this is by clicking `Runtime -> Run All` in the notebook menu.

##### *Policy regarding Google Colab:*
- The instruction in this notebook assumes that you will use Colab.

- However, using Colab is not required. You are free to run the code on your local machine, though in that case you may suffer from a slow runtime due to the lack of proper GPU resources.

---

##### *Academic honesty*

- We will audit the notebooks from a set number of students, chosen at
  random. The audits will check that the code you wrote actually generates the
  answers in your report PDF. If you turn in correct answers on your PDF without code
  that actually generates those answers, we will consider this a serious case of
  cheating. See the course page for honesty policies.

- We will also run automatic checks of notebooks for plagiarism.
  Copying code from others is also considered a serious case of cheating.

---

# Checkpoint 1: Data Collection and Annotation

In this homework, you will first collect a labeled dataset of **150** sentences for a text classification task of your choice. This process will include:

1. *Data collection*: Collect 150 sentences from any source you find interesting (e.g., literature, Tweets, news articles, reviews, etc.)

2. *Task design*: Come up with a multilabel sentence-level classification task that you would like to perform on your sentences.

3. On your dataset, collect annotations from **two** classmates for your task on a **second, separate set** of a minimum of **150** sentences. Everyone in this class will need to both create their own dataset and also serve as an annotator for two other classmates. In order to get everything done on time, you need to complete the following steps:

> *   Find two classmates willing to label 150 sentences each (use the Piazza "search for teammates" thread if you're having issues finding labelers).
*   Collect the labeled data from each of the two annotators.
*   Sanity check the data for basic cleanliness (are all examples annotated? are all labels allowable ones?)

4. Collect feedback from annotators about the task including annotation time and obstacles encountered (e.g., maybe some sentences were particularly hard to annotate!)

5. Calculate and report inter-annotator agreement.

6. Aggregate output from both annotators to create final dataset (include your first 150 sentences too).

7. Perform NLP experiments on your new dataset!

The mapping of label names and IDs in seed.tsv is as follows:

```json
{
    'Economic': 1.0,
    'Capacity and Resources': 2.0,
    'Morality': 3.0,
    'Fairness and Equality': 4.0,
    'Legality, Constitutionality, Jurisdiction': 5.0,
    'Policy Prescription and Evaluation': 6.0,
    'Crime and Punishment': 7.0,
    'Security and Defense': 8.0,
    'Health and Safety': 9.0,
    'Quality of Life': 10.0,
    'Cultural Identity': 11.0,
    'Public Sentiment': 12.0,
    'Political': 13.0,
    'External Regulation and Reputation': 14.0,
    'Other': 15.0
}
```

Make sure that this mapping is followed in all of your data files.

## Question 3 (8 points):
Now, compute the inter-annotator agreement between your two annotators. Upload both .tsv files to your Colab session (click the folder icon in the sidebar to the left of the screen). In the code cell below, read the data from the two files and compute both the raw agreement (% of examples for which both annotators agreed on the label) and the [Cohen's Kappa](https://en.wikipedia.org/wiki/Cohen%27s_kappa). Feel free to use implementations in existing libraries (e.g., [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html)). After you’re done, report the raw agreement and Cohen’s scores in your report.

*If you're curious, Cohen suggested the Kappa result be interpreted as follows: values ≤ 0 as indicating no agreement and 0.01–0.20 as none to slight, 0.21–0.40 as fair, 0.41– 0.60 as moderate, 0.61–0.80 as substantial, and 0.81–1.00 as almost perfect agreement.*

In [ ]:
### WRITE CODE TO LOAD ANNOTATIONS AND
### COMPUTE AGREEMENT + COHEN'S KAPPA HERE!
import pandas as pd
from sklearn.metrics import cohen_kappa_score

def raw_agreement(annotator1_df, annotator2_df):
    # TODO: Implement this function
    df1 = pd.read_csv(annotator1_df, sep='\t')
    df2 = pd.read_csv(annotator2_df, sep='\t')
    label1 =  df1['label']
    label2 =  df2['label']

    count_same = 0
    for i in range(len(label1)):
        if(label1[i] == label2[i]):
            count_same = count_same+1


    return count_same/len(label1)

def kohens_cappa(annotator1_df, annotator2_df):
    # TODO: Implement this function
    df1 = pd.read_csv(annotator1_df, sep='\t')
    df2 = pd.read_csv(annotator2_df, sep='\t')
    label1 =  df1['label']
    label2 =  df2['label']
    return cohen_kappa_score(label1,label2)



# TODO: load your data correctly
annotator1_df = "annotator1.tsv"
annotator2_df = "annotator2.tsv"

print("--- Raw agreement between annotator1 and annotator2 ---")
print(raw_agreement(annotator1_df, annotator2_df))

print("--- Cohen's kappa score between annotator1 and annotator2 ---")
print(kohens_cappa(annotator1_df, annotator2_df))

TODO : Write the values obtained above in this cell.

### *RAW AGREEMENT*:  0.7577639751552795

### *COHEN'S KAPPA*:  0.7395578414699905

# Checkpoint 2: Model Training and Testing

Now we'll move onto fine-tuning  pretrained language models specifically on your dataset. This part of the homework is meant to be an introduction to the HuggingFace library, and it contains code that will potentially be useful for your final projects. Since we're dealing with large models, the first step is to change to a GPU runtime.

## Adding a hardware accelerator

Please go to the menu and add a GPU as follows:

`Edit > Notebook Settings > Hardware accelerator > (GPU)`

Run the following cell to confirm that the GPU is detected.

In [1]:
import torch
torch.cuda.empty_cache()

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


In [2]:
import random
import numpy as np
import torch

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_everything()

## Installing Hugging Face's Transformers library
We will use Hugging Face's Transformers (https://github.com/huggingface/transformers), an open-source library that provides general-purpose architectures for natural language understanding and generation with a collection of various pretrained models made by the NLP community. This library will allow us to easily use pretrained models like `BERT` and perform experiments on top of them. We can use these models to solve downstream target tasks, such as text classification, question answering, and sequence labeling.

Run the following cell to install Hugging Face's Transformers library and download a sample data file called seed.tsv that contains 250 sentences in English, annotated with their frame.

In [4]:
!pip install transformers
!pip install -U -q PyDrive

The cell below imports some helper functions we wrote to demonstrate the task on
the sample seed dataset.

#### *IMPORTANT NOTE*:

The tokenize_and_format function in helpers.py uses bert-base-uncased as the
model for the tokenizer. If you are using a different model for training in this
notebook or for running predictions in a different notebook or python file, you
need to change the model name as well in the tokenizer, otherwise you will get
arbitrarily incorrect results down the line.

If you update the model name for the tokenizer, you would need to reload the
file which can be done simply by re-running the cell below.

In [5]:
from helpers import tokenize_and_format, flat_accuracy

# Data Prep and Model Specifications

Upload your data using the file explorer to the left. We have provided a
function below to tokenize and format your data as BERT requires. Make sure that
your tsv file, titled final_data.tsv, has one column "sentence" and another
column "label_ID" containing integers/float. (basically the same format as
seed.tsv should be maintained for the sentence and label columns)

If you run the cell below without modifications, it will run on the seed.tsv
example data we have provided. It imports some helper functions we wrote to
demonstrate the task on the sample dataset. You should first run all of the
following cells with seed.tsv just to see how everything works. Then, once you
understand the whole preprocessing / fine-tuning process, change the tsv in the
below cell to your final_data.tsv file, add any extra preprocessing code you
wish, and then run the cells again on your own data.


#### Important Note :

The code below expects the data to be in a tsv file  with the columns as "sentence"
and "label_ID" (other columns are not that relevant here). But this is different
from the instructions in the report where you are expected to create data with
"text" and "label" columns for all of the annotation steps.

Modify the code below to suitably handle this.

In [46]:
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
import numpy as np

seed_everything()

#df = pd.read_csv('final_data.tsv') # TODO : Uncomment this line to use the full dataset
df = pd.read_csv('dtangira_final_data.tsv',sep='\t')

df = df.sample(frac=1).reset_index(drop=True)
texts = df.text.values # this assumes that the column containing the text is called "sentence"
labels = df.label.values # this assumes that the column containing the labels is called "label_ID"
temp_labels = labels # Store the Labels before converting into

### tokenize_and_format() is a helper function provided in helpers.py ###
### Male sure you use the correct model name for your tokenizer! ###
input_ids, attention_masks = tokenize_and_format(texts)

label_list = []
for l in labels:
  label_array = np.zeros(len(set(labels)))
  label_array[int(l)-1] = 1
  label_list.append(label_array)

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(np.array(label_list))

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])

Original:  Legal recognition of same-sex marriage contributes to the health and safety of LGBTQ couples by providing access to crucial healthcare benefits, including insurance coverage for spouses.
Token IDs: tensor([  101,  3423,  5038,  1997,  2168,  1011,  3348,  3510, 16605,  2000,
         1996,  2740,  1998,  3808,  1997, 12010,  4160,  6062,  2011,  4346,
         3229,  2000, 10232,  9871,  6666,  1010,  2164,  5427,  6325,  2005,
        18591,  2015,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])


## Create train/test/validation splits

Here we split your dataset into 3 parts: a training set, a validation set, and a testing set. Each item in your dataset will be a 3-tuple containing an input_id tensor, an attention_mask tensor, and a label tensor.



In [47]:
seed_everything()

total = len(df)

num_train = int(total * .8)
num_val = int(total * .1)
num_test = total - num_train - num_val

# make lists of 3-tuples (already shuffled the dataframe in cell above)
train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
val_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train, num_val+num_train)]
test_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_val + num_train, total)]

train_text = [texts[i] for i in range(num_train)]
train_label = [temp_labels[i] for i in range(num_train)]

val_text = [texts[i] for i in range(num_train, num_val+num_train)]
test_text = [texts[i] for i in range(num_val + num_train, total)]



# Data Prep and Model Specifications for New Multilingual Data *****

In [57]:
from helpers import tokenize_and_format, flat_accuracy
import pandas as pd
import numpy as np

seed_everything()

#df = pd.read_csv('final_data.tsv') # TODO : Uncomment this line to use the full dataset
df_new = pd.read_csv('dtangira_augmented_data.tsv',sep='\t')

df_new = df_new.sample(frac=1).reset_index(drop=True)
texts = df_new.text.values # this assumes that the column containing the text is called "sentence"
labels = df_new.label.values # this assumes that the column containing the labels is called "label_ID"
#temp_labels = labels # Store the Labels before converting into
print(len(labels))

### tokenize_and_format() is a helper function provided in helpers.py ###
### Male sure you use the correct model name for your tokenizer! ###
input_ids, attention_masks = tokenize_and_format(texts)

label_list = []
for l in labels:
  label_array = np.zeros(len(set(labels)))
  label_array[int(l)-1] = 1
  label_list.append(label_array)

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(np.array(label_list))

# Print sentence 0, now as a list of IDs.
print('Original: ', texts[0])
print('Token IDs:', input_ids[0])

4016
Original:  धार्मिकता की भावना से प्रेरित होकर, धार्मिक समुदाय आप्रवासियों के अधिकारों का समर्थन करने और मानवीय गरिमा को बनाए रखने वाले व्यापक आप्रवासन सुधार की वकालत करने के लिए संगठित हो सकते हैं।
Token IDs: tensor([  101,  1326, 29876, 29869, 29867, 29877, 29851, 29859, 29876,  1315,
        29878,  1330, 29876, 29871, 29863, 29876,  1338,  1328, 29869, 29869,
        29877, 29859,  1339, 29879, 29851, 29869,  1010,  1326, 29876, 29869,
        29867, 29877, 29851,  1338, 29867, 29861, 29876, 29868,  1312, 29864,
        29869, 29871, 29876, 29874, 29877, 29868, 29879,  1315,  1311, 29862,
        29877, 29851, 29876, 29869, 29879,  1315, 29876,  1338, 29867, 29869,
        29860, 29863,  1315,   102])


# Create train splits for Multilingual Data ****

In [58]:
seed_everything()
num_train = len(df_new)

train_set = [(input_ids[i], attention_masks[i], labels[i]) for i in range(num_train)]
train_text = [texts[i] for i in range(num_train)]

In [59]:
print(len(train_set))
print(len(test_set))

4016
32


Here we choose the model we want to finetune from https://huggingface.co/transformers/pretrained_models.html. Because the task requires us to label sentences, we wil be using BertForSequenceClassification below. You may see a warning that states that `some weights of the model checkpoint at [model name] were not used when initializing. . .` This warning is expected and means that you should fine-tune your pre-trained model before using it on your downstream task. See [here](https://github.com/huggingface/transformers/issues/5421#issuecomment-652582854) for more info.

In [60]:
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification, BertConfig
from torch.optim import AdamW

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer English BERT model, with an uncased vocab.
    num_labels = 15, # The number of output labels.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

# Choose a MODEL for BERT ****

In [ ]:
from transformers import BertForSequenceClassification, AutoModelForSequenceClassification, BertConfig
from torch.optim import AdamW

model = BertForSequenceClassification.from_pretrained(
    "bert-base-multilingual-cased", # Use model of the top 104 languages with the largest Wikipedia
    num_labels = 15, # The number of output labels.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OutOfMemoryError: CUDA out of memory. Tried to allocate 352.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 87.06 MiB is free. Process 4783 has 14.66 GiB memory in use. Of the allocated memory 14.11 GiB is allocated by PyTorch, and 426.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# TODO: ACTION REQUIRED #

Define your fine-tuning hyperparameters in the cell below (we have randomly picked some values to start with). We want you to experiment with different configurations to find the one that works best (i.e., highest accuracy) on your validation set. Feel free to also change pretrained models to others available in the HuggingFace library (you'll have to modify the cell above to do this). You might find papers on BERT fine-tuning stability (e.g., [Mosbach et al., ICLR 2021](https://openreview.net/pdf?id=nzpLWnVAyah)) to be of interest.

In [ ]:
batch_size = 2
# you can change lr and eps values in the AdamW call if you like
optimizer = AdamW(model.parameters(),lr=5e-5,eps=1e-6) #with default values of learning rate and epsilon value
epochs =  16

# Choose hyper parameters for Multilingual Model ****

In [61]:
batch_size = 64
# you can change lr and eps values in the AdamW call if you like
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-5) #with default values of learning rate and epsilon value
epochs =  50

batch_size = 64
# you can change lr and eps values in the AdamW call if you like
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-5) #with default values of learning rate and epsilon value
epochs =  60


# Fine-tune your model
Here we provide code for fine-tuning your model, monitoring the loss, and checking your validation accuracy. Rerun both of the below cells when you change your hyperparameters above.

In [38]:
# function to get validation accuracy
def get_validation_performance(val_set):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables
    total_eval_accuracy = 0
    total_eval_loss = 0

    num_batches = int(len(val_set)/batch_size) + 1

    total_correct = 0

    for i in range(num_batches):

      end_index = min(batch_size * (i+1), len(val_set))

      batch = val_set[i*batch_size:end_index]

      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)

      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():

        # Forward pass, calculate logit predictions.
        # Note: this line of code might need to change depending on the model
        # the current line will work for bert-base-uncased
        # please refer to huggingface documentation for other models
        outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
        loss = outputs.loss
        logits = outputs.logits

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = (logits).detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()


        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = np.argmax(label_ids, axis=1).flatten()

        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct

    # Report the final accuracy for this validation run.
    print("Num of correct predictions =", total_correct)
    avg_val_accuracy = total_correct / len(val_set)
    return avg_val_accuracy



In [62]:
import random
seed_everything()

# training loop


# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    num_batches = int(len(train_set)/batch_size) + 1

    for i in range(num_batches):
      end_index = min(batch_size * (i+1), len(train_set))

      batch = train_set[i*batch_size:end_index]

      if len(batch) == 0: continue

      input_id_tensors = torch.stack([data[0] for data in batch])
      input_mask_tensors = torch.stack([data[1] for data in batch])
      label_tensors = torch.stack([data[2] for data in batch])

      # Move tensors to the GPU
      b_input_ids = input_id_tensors.to(device)
      b_input_mask = input_mask_tensors.to(device)
      b_labels = label_tensors.to(device)

      optimizer.zero_grad()

      # Perform a forward pass (evaluate the model on this training batch).
      # this line of code might need to change depending on the model
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc = get_validation_performance(val_set)
    print(f"Validation accuracy: {val_acc}")

print("")
print("Training complete!")




======== Epoch 1 / 60 ========
Training...
Total loss: 27.36504389930893
Num of correct predictions = 0
Validation accuracy: 0.0

======== Epoch 2 / 60 ========
Training...
Total loss: 16.277714449940763
Num of correct predictions = 2
Validation accuracy: 0.06451612903225806

======== Epoch 3 / 60 ========
Training...
Total loss: 15.494554685697794
Num of correct predictions = 2
Validation accuracy: 0.06451612903225806

======== Epoch 4 / 60 ========
Training...
Total loss: 15.3880233369315
Num of correct predictions = 2
Validation accuracy: 0.06451612903225806

======== Epoch 5 / 60 ========
Training...
Total loss: 15.381063232956354
Num of correct predictions = 2
Validation accuracy: 0.06451612903225806

======== Epoch 6 / 60 ========
Training...
Total loss: 15.372192070238656
Num of correct predictions = 5
Validation accuracy: 0.16129032258064516

======== Epoch 7 / 60 ========
Training...
Total loss: 15.33553467752386
Num of correct predictions = 5
Validation accuracy: 0.161290322

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TODO: SAVE YOUR MODEL HERE... (Refer PyTorch documentation for how to save models)
torch.save(model.state_dict(),"/content/drive/MyDrive/Colab_Notebooks/Best.pt")

In [ ]:
# SAVE NEW MULTILINGUAL MODEL...
torch.save(model.state_dict(),"/content/drive/MyDrive/Colab_Notebooks/Multi_Best.pt")

# Evaluate your model on the test set
After you're satisfied with your hyperparameters (i.e., you're unable to achieve higher validation accuracy by modifying them further), it's time to evaluate your model on the test set! Run the below cell to compute test set accuracy.


In [63]:
seed_everything()

# If your notebook disconnects during training, then here, first load the best
# model you saved (refer PyTorch docs), then check validation performance

#model.load_state_dict(torch.load("/content/drive/MyDrive/Colab_Notebooks/Best.pt"))
get_validation_performance(test_set)

Num of correct predictions = 19


0.59375

# Evaluate your Multi-Lingual model on the test set ****

In [ ]:
seed_everything()

# If your notebook disconnects during training, then here, first load the best
# model you saved (refer PyTorch docs), then check validation performance
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab_Notebooks/Multi_Best.pt"))
get_validation_performance(test_set)

Num of correct predictions = 25


0.78125

## Question 8 (10 points):
Finally, perform an *error analysis* on your model. This is good practice for your final project. Write some code in the below code cell to print out the text of up to five test set examples that your model gets **wrong**. If your model gets more than five test examples wrong, randomly choose five of them to analyze. If your model gets fewer than five examples wrong, please design five test examples that fool your model (i.e., *adversarial examples*). Then, in the following text cell, perform a qualitative analysis of these examples. See if you can figure out any reasons for errors that you observe, or if you have any informed guesses (e.g., common linguistic properties of these particular examples). Does this analysis suggest any possible future steps to improve your classifier?

In [ ]:
## YOUR ERROR ANALYSIS CODE HERE

## print out up to 5 test set examples (or adversarial examples) that your model gets wrong
seed_everything()
torch.cuda.empty_cache()
model.load_state_dict(torch.load("/content/drive/MyDrive/Colab_Notebooks/Best.pt"))
#get_validation_performance(test_set)
model.eval()

# Tracking variables
total_eval_accuracy = 0
total_eval_loss = 0

num_batches = int(len(test_set)/batch_size) + 1

total_correct = 0
error_pred = []
for i in range(num_batches):
  end_index = min(batch_size * (i+1), len(test_set))
  batch = test_set[i*batch_size:end_index]
  if len(batch) == 0: continue
  input_id_tensors = torch.stack([data[0] for data in batch])
  input_mask_tensors = torch.stack([data[1] for data in batch])
  label_tensors = torch.stack([data[2] for data in batch])

  # Move tensors to the GPU
  b_input_ids = input_id_tensors.to(device)
  b_input_mask = input_mask_tensors.to(device)
  b_labels = label_tensors.to(device)

  # Tell pytorch not to bother with constructing the compute graph during
  # the forward pass, since this is only needed for backprop (training).
  with torch.no_grad():


    # Forward pass, calculate logit predictions.
    # Note: this line of code might need to change depending on the model
    # the current line will work for bert-base-uncased
    # please refer to huggingface documentation for other models
    outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,
                                labels=b_labels)
    loss = outputs.loss
    logits = outputs.logits

    # Accumulate the validation loss.
    total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = (logits).detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the number of correctly labeled examples in batch
    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = np.argmax(label_ids, axis=1).flatten()
    for ind in range(len(batch)):
      if pred_flat[ind] != labels_flat[ind]:
        error_pred.append((test_text[ind], pred_flat[ind], labels_flat[ind]))

    num_correct = np.sum(pred_flat == labels_flat)
    total_correct += num_correct



for i in range(5):
  print(error_pred[i][0])
  print("Predicted Label",error_pred[i][1])
  print("Correct Label",error_pred[i][2])



Advocates for immigration equality argue for policies that prioritize human rights, dignity, and inclusivity, recognizing the contributions and humanity of all individuals regardless of immigration status.
Predicted Label 10
Correct Label 3
In recent decades, immigration patterns have led to more cross-cultural experiences on both sides of the border.
Predicted Label 9
Correct Label 13
The costs of immigration can include expenditures on public services such as education, healthcare, and social welfare programs for immigrants and their families.
Predicted Label 0
Correct Label 9
Limited access to legal representation for migrants, particularly those facing deportation proceedings, undermines due process and fairness within the immigration system.
Predicted Label 3
Correct Label 1
Marriage equality grants same-sex couples legal protections against domestic violence and abuse, previously more difficult to address.
Predicted Label 5
Correct Label 6


In [3]:
pip install googletrans==3.1.0a0


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 12.2 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=1d31206c20b9f7f039ab916672225ca0790da81604a36038171c4e0a3609737f
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling

# Translation for seed.tsv and training data

In [33]:
## Install googletrans first using one of the methods below:
# pip install googletrans==4.0.0-rc1
# pip install googletrans==3.1.0a0
# pip install google_trans_new
seed_everything()
import googletrans
from googletrans import Translator
import pandas as pd
import time
# see available languages with the below
print(googletrans.LANGUAGES)

# Init
translator = Translator()

# Call to translate and get output
output = translator.translate("I love natural language processing", src='en', dest='fr')
#print(output.text)
df = pd.read_csv('seed_New - seed.tsv', sep='\t')

#df = pd.read_csv('dtangira_final_data.tsv', sep='\t')

print(len(list(df['label_ID']) ))
print (len(train_text))
label =  train_label + list(df['label_ID']) # Only use training set label and Seed.tsv data

#print(train_label)
text = train_text + list(df['sentence']) # Only use training set data and seed.tsv data
#language = df['language']
print (len(label))
print (len(text))
all_lang = ['en','zh-cn','hi','te','bn','el','ru','tr']

columns = ['text', 'language','label','source']
new_text =[]
new_language = []
new_label = []
new_source = []
new_annotator = []
length = len(text)
for i in all_lang:
    print(i)
    if(i == 'en'):
        for line in range(length):
            #time.sleep(1)
            temp = translator.translate(text[line], src='en',dest='te')
            new_text.append((translator.translate(temp.text, src='te',dest='en')).text)
            new_language.append(i)
            new_source.append('en'+'->'+'en')
            new_label.append(label[line])

    else:
        for line in range(length):

            #temp = translator.translate(text[line],src='eng',dest='hin')

            new_text.append((translator.translate(text[line],src='en',dest=i)).text)
            new_language.append(i)
            new_source.append('en' + '->' + i)
            new_label.append(label[line])
data = {}
data['text'] = new_text
data['language'] = new_language
data['label'] = new_label
data['source'] = new_source
df = pd.DataFrame(data=data)
output_file = 'dtangira_augmented_data.tsv'

df.to_csv(output_file, sep='\t', index=False)


{'af': 'afrikaans', 'sq': 'albanian', 'am': 'amharic', 'ar': 'arabic', 'hy': 'armenian', 'az': 'azerbaijani', 'eu': 'basque', 'be': 'belarusian', 'bn': 'bengali', 'bs': 'bosnian', 'bg': 'bulgarian', 'ca': 'catalan', 'ceb': 'cebuano', 'ny': 'chichewa', 'zh-cn': 'chinese (simplified)', 'zh-tw': 'chinese (traditional)', 'co': 'corsican', 'hr': 'croatian', 'cs': 'czech', 'da': 'danish', 'nl': 'dutch', 'en': 'english', 'eo': 'esperanto', 'et': 'estonian', 'tl': 'filipino', 'fi': 'finnish', 'fr': 'french', 'fy': 'frisian', 'gl': 'galician', 'ka': 'georgian', 'de': 'german', 'el': 'greek', 'gu': 'gujarati', 'ht': 'haitian creole', 'ha': 'hausa', 'haw': 'hawaiian', 'iw': 'hebrew', 'he': 'hebrew', 'hi': 'hindi', 'hmn': 'hmong', 'hu': 'hungarian', 'is': 'icelandic', 'ig': 'igbo', 'id': 'indonesian', 'ga': 'irish', 'it': 'italian', 'ja': 'japanese', 'jw': 'javanese', 'kn': 'kannada', 'kk': 'kazakh', 'km': 'khmer', 'ko': 'korean', 'ku': 'kurdish (kurmanji)', 'ky': 'kyrgyz', 'lo': 'lao', 'la': 'lat

# Predictions from JSON ****

In [64]:
import json

seed_everything()
torch.cuda.empty_cache()

#model.load_state_dict(torch.load("/content/drive/MyDrive/Colab_Notebooks/Multi_Best.pt"))
#model.eval()
id_mapping = {0: 'None', 1: 'Economic', 2: 'Capacity and Resources', 3: 'Morality', 4: 'Fairness and Equality', 5: 'Legality, Constitutionality, Jurisdiction', 6: 'Policy Prescription and Evaluation', 7: 'Crime and Punishment', 8: 'Security and Defense', 9: 'Health and Safety', 10: 'Quality of Life', 11: 'Cultural Identity', 12: 'Public Sentiment', 13: 'Political', 14: 'External Regulation and Reputation', 15: 'Other'}


# Tracking variables
total_eval_accuracy = 0
total_eval_loss = 0

with open("test_set.json", "r") as f:
    test_set = json.load(f)


num_batches = int(len(test_set)/batch_size) + 1

total_correct = 0

for i in range(num_batches):
  end_index = min(batch_size * (i+1), len(test_set))
  batch = test_set[i*batch_size:end_index]
  if len(batch) == 0: continue
  texts = []
  for data in batch:
    texts.append(data['sentence'])
  input_ids, attention_masks = tokenize_and_format(texts)

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  # Move tensors to the GPU
  b_input_ids = input_ids.to(device)
  b_input_mask = attention_masks.to(device)
  #b_labels = label_tensors.to(device)

  # Tell pytorch not to bother with constructing the compute graph during
  # the forward pass, since this is only needed for backprop (training).
  with torch.no_grad():


    # Forward pass, calculate logit predictions.
    # Note: this line of code might need to change depending on the model
    # the current line will work for bert-base-uncased
    # please refer to huggingface documentation for other models
    outputs = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask)
    loss = outputs.loss
    logits = outputs.logits
    error_pred = []
    # Accumulate the validation loss.
    #total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = (logits).detach().cpu().numpy()
    # Calculate the number of correctly labeled examples in batch
    pred_flat = np.argmax(logits, axis=1).flatten()
    for ind in range(len(pred_flat)):
      test_set[ind + (i*batch_size)]['label_ID'] = str(pred_flat[ind])
      test_set[ind + (i*batch_size)]['predicted_label'] = id_mapping[pred_flat[ind]]


with open("dtangira.json", "w") as f:
    json.dump(test_set, f, ensure_ascii=False, indent=4)






### *DESCRIBE YOUR QUALITATIVE ANALYSIS OF THE ABOVE EXAMPLES IN YOUR REPORT*